<a href="https://colab.research.google.com/github/ddebashish21/Covid-prediction/blob/main/Covid_Prediction_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Image Classification for the prediction of Covid-19 using lung's X-ray**

The dataset contains images of the lung's X-ray of normal people and the lung's X-ray of Covid-19 patients. The aim of the project is to train a model to distinguise the covid-19 patients from the normal patients using the X-ray images.

In [ ]:
# Importing all the essential libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import cv2

In [ ]:
# Creating the directories for storing image datas
mkdir covid_dataset

In [ ]:
mkdir covid_dataset/train

In [ ]:
mkdir covid_dataset/test

In [ ]:
mkdir covid_dataset/train/covid

In [ ]:
mkdir covid_dataset/train/normal

In [ ]:
mkdir covid_dataset/test/covid

In [ ]:
mkdir covid_dataset/test/normal

In [ ]:
# Storing the address of training and testing datas
ref={"train": "/content/covid_dataset/train",
     "test": "/content/covid_dataset/test"}

**Preprocessing the image dataset for training**

In [ ]:
# Preprocessing the training data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

In [ ]:
# Preprocessing the testing data
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
training_data = train_datagen.flow_from_directory(ref["train"],
                                                 target_size=(64, 64),
                                                 batch_size=32,
                                                 class_mode='binary')

Found 181 images belonging to 2 classes.


In [ ]:
testing_data = test_datagen.flow_from_directory(ref["test"],
                                               target_size=(64, 64),
                                               batch_size=32,
                                               class_mode='binary')

Found 46 images belonging to 2 classes.


In [ ]:
print(training_data.class_indices)
print(testing_data.class_indices)

{'covid': 0, 'normal': 1}
{'covid': 0, 'normal': 1}


In [ ]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(64,64,3))) # 1st Convolution layer
model.add(MaxPooling2D(pool_size=(2,2))) # 1st Pooling layer using the maximum value
model.add(Conv2D(16,(3,3),activation='relu')) # 2nd Convokution layer
model.add(MaxPooling2D(pool_size=(2,2))) # 2nd Pooling layer using the maximum value
model.add(Flatten()) # Fully-connected layer

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Adding 3 hidden layers of Neural network
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=64,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))

In [ ]:
# Compiling the model using accuracy metrics
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
# Training the model using the training dataset and validating it using testing dataset
model.fit(training_data,
          steps_per_epoch=int(len(training_data)/32),
          epochs=5,
          validation_data=testing_data,
          validation_steps=int(len(testing_data)/32))

Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 324ms/step - accuracy: 0.9177 - loss: 0.2009 - val_accuracy: 0.9565 - val_loss: 0.1418
Epoch 2/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 206ms/step - accuracy: 0.9335 - loss: 0.1570 - val_accuracy: 0.9565 - val_loss: 0.1479
Epoch 3/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 215ms/step - accuracy: 0.8973 - loss: 0.1844 - val_accuracy: 0.9565 - val_loss: 0.1375
Epoch 4/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 361ms/step - accuracy: 0.9140 - loss: 0.2003 - val_accuracy: 0.9565 - val_loss: 0.1372
Epoch 5/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 208ms/step - accuracy: 0.9032 - loss: 0.1617 - val_accuracy: 0.9348 - val_loss: 0.1685


**Testing the models accuracy using images manually**

In [ ]:
# importing the necessary libraries
import keras.preprocessing.image as image
import numpy as np

In [ ]:
# Setting the dimension of the image according to the model requirements
img1=image.load_img('/content/covid_dataset/test/covid/COVID-00003b.jpg',target_size=(64,64))
img2=image.load_img('/content/covid_dataset/test/normal/0105.jpeg',target_size=(64,64))

In [ ]:
# Normalizing the pixels and expanding the size of the image
test_img1=image.img_to_array(img1)
test_img2=image.img_to_array(img2)
test_img1=test_img1/255.
test_img2=test_img2/255.
test_img1=np.expand_dims(test_img1,axis=0)
test_img2=np.expand_dims(test_img2,axis=0)

In [ ]:
# Defining the classes with the respective name
res={0: "Covid",
     1: "Normal"}

In [ ]:
print(res[round(model.predict(test_img1)[0][0])])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Covid


In [ ]:
print(res[round(model.predict(test_img2)[0][0])])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Normal


After running the model through all 5 epochs we had set, the accuracy comes around 90.32% and the loss is around 16.17 and when the model is compared with the testing data the accuracy comes around 93.48% and the loss comes around 16.85%. We had further tried to take two randon images from the dataset and fed the model for prediction and the model has rightfully predicted the outcome.